## Step1.Seperate isolated nodes and non-isolated nodes from original graph

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(NAIR)
library(tidyverse)
library(tictoc)
library(igraph)
library(pryr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Welcome to NAIR: Network Analysis of Immune Repertoire.
Get started using `vignette("NAIR")`, or by visiting
https://mlizhangx.github.io/Network-Analysis-for-Repertoire-Sequencing-/

次のパッケージを付け加えます: ‘igraph’

以下のオブジェクトは ‘package:lubridate’ からマスクされています:

    %--%, union

以下のオブジェクトは ‘package:dplyr’ からマスクされています:

    as_data_frame, groups, union

以下のオブジェクトは ‘package:purrr’ からマスクされています:

    compose, simplify

以下のオブジェクトは ‘package:tidyr’ からマスクされています:

    crossing

以下のオブジェクトは ‘package:tibble’ からマスクされています:

    as_data_frame

以下のオブジェクトは ‘package:stats’ からマスクされています:

    decompose, spectrum

以下のオブジェクトは ‘package:base’ からマスクされています:

    union


次のパッケージを付け加えます: ‘pryr’

以下のオブジェクトは ‘package:igraph’ からマスクされています:

    compose

以下のオブジェクトは ‘package:dplyr’ からマスクされています:

    where

以下のオブジェクトは ‘package:purrr’ からマスクされています:

    compose, partial



In [3]:
%%R

create_igraph_from_mtx_file <- function(directory, file_name) {
  ## file_name : directory_to_the_folder/txt_file_name.mtx
  ## 
  mtxfile <- Matrix::readMM(file.path(directory, file_name))
  igraph_object <- graph_from_adjacency_matrix(mtxfile, mode=c("undirected")) %>%##graph_from_adjacency_matrix is a function from igraph
    set_vertex_attr(name = "name", value = seq_along(V(.))) 
  return(igraph_object)
}

##------------------------------------------------------------------------------
## Write igraph_object data into txt file without reset indexes-----------------
##------------------------------------------------------------------------------
write_isolated_nonisolated_subgraph_dat <- function(igraph_object, directory) {
  ## In: igraph_object: an igraph object
  ## Out: a data frame that contains the data of the non_isolated_subgraph with indexes gotten from the original igraph_object
  ##      the indexes are set as character to prevent igraph to reset the index, so that we can keep their original values
  isolated_node_l <- which(degree(igraph_object, loops = FALSE)==0)   ## Find isolated node index
  ##
  isolated_subgraph_dat <- igraph_object %>% igraph::as_data_frame() %>%  filter(from %in% isolated_node_l) %>% select(c("from"))
  non_isolated_subgraph_dat <- igraph_object %>% igraph::as_data_frame() %>%  filter(!from %in% isolated_node_l) 
  ##
  write.table(isolated_subgraph_dat, file.path(directory, "isolated_subgraph_dat.txt"), row.names = F, col.names = F)
  write.table(non_isolated_subgraph_dat, file.path(directory, "non_isolated_subgraph_dat.txt"), row.names = F, col.names = F)
}

org_igraph <- create_igraph_from_mtx_file("./Original", "MyRepSeqNetwork_AdjacencyMatrix.mtx")
write_isolated_nonisolated_subgraph_dat(org_igraph, "./Original")

In [ ]:
import sys
sys.path.append('/Users/awu10/Desktop/TCR_Subsampling/Pre/Code/PythonCode/CleanVersion')  #  path to the code location
import Graph_Sampling
from scipy.io import mmread
import networkx as nx
import numpy as np
import random
import os
import math
import pandas as pd
import helper

In [6]:
# ## Step 1: Set up needed arguments
## For input directory - the original graph----------------
input_dir = "./"

input_file_name = "MyRepSeqNetwork_AdjacencyMatrix.mtx" ## the input mtx file from NAIR

## For output directory - the subgraph 
all_sub_dir = "./subgraph"
 
# ## Step 2: Get the original network from AdjacencynMatrix-----------------------
g= mmread(os.path.join(input_dir, input_file_name))
org_graph = nx.Graph(g)
helper.graph_glimpse(org_graph) # NODE: the node_indexes of the graph from NAIR mtx

Number of nodes:  87860
Number of edges:  181507
Some first nodes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Some first edges:  [(0, 0), (0, 89), (0, 110), (0, 397), (0, 1194), (0, 1335), (0, 1338), (0, 1339), (0, 1500), (0, 1501), (0, 1532), (0, 1535), (0, 1720), (0, 1746), (0, 1748), (0, 1788), (0, 1872), (0, 1972), (0, 2006), (0, 2722), (0, 3456), (0, 4143), (0, 4183), (0, 4187), (0, 4768), (0, 5374), (0, 7470), (0, 8952), (0, 8953), (0, 10952), (0, 11078), (0, 14531), (0, 21089), (0, 28478), (0, 29546), (0, 29633), (0, 44513), (0, 44761), (0, 44867), (0, 47773), (0, 47845), (0, 48085), (0, 48086), (0, 48087), (0, 48088), (0, 48089), (0, 4

In [7]:
##batch sampling usng non-induced method
sample_size_percent_l = [5, 10, 15, 20, 25, 30] 
methods = ["MH", "PR", "SB", "RNS","SRWFB"]
for method in methods:
    person_sub_out_dir = os.path.join(all_sub_dir, method)
    print(method)
    os.makedirs(person_sub_out_dir, exist_ok=True)
    helper.do_sampling_for_a_sample_percent_list(org_graph, sample_size_percent_l, 
                                                 percent_indi = True,  
                                                 sub_method = method,
                                                 output_dir = person_sub_out_dir, 
                                                 induce= False)

MH
sub-graph's size:  4393
Number of nodes: 776
sub-graph's size:  8786
Number of nodes: 114
sub-graph's size:  13179
Number of nodes: 1
sub-graph's size:  17572
Number of nodes: 1
sub-graph's size:  21965
Number of nodes: 501
sub-graph's size:  26358
Number of nodes: 1
PR
sub-graph's size:  4393
Number of nodes: 4393
sub-graph's size:  8786
Number of nodes: 8786
sub-graph's size:  13179
Number of nodes: 13179
sub-graph's size:  17572
Number of nodes: 17572
sub-graph's size:  21965
Number of nodes: 21965
sub-graph's size:  26358
Number of nodes: 26358
SB
sub-graph's size:  4393
Number of nodes: 4396
sub-graph's size:  8786
Number of nodes: 8788
sub-graph's size:  13179
Number of nodes: 13185
sub-graph's size:  17572
Number of nodes: 17574
sub-graph's size:  21965
Number of nodes: 21967
sub-graph's size:  26358
Number of nodes: 26359
RNS
sub-graph's size:  4393
Number of nodes: 4393
sub-graph's size:  8786
Number of nodes: 8786
sub-graph's size:  13179
Number of nodes: 13179
sub-graph's

In [8]:
##batch sampling usng induced method
sample_size_percent_l = [5, 10, 15, 20, 25, 30] 
methods = ["InPR", "InMH","InSRWFB"]
for method in methods:
    person_sub_out_dir = os.path.join(all_sub_dir, method)
    print(method)
    os.makedirs(person_sub_out_dir, exist_ok=True)
    helper.do_sampling_for_a_sample_percent_list(org_graph, sample_size_percent_l, 
                                                 percent_indi = True,  
                                                 sub_method = method,
                                                 output_dir = person_sub_out_dir, 
                                                 induce= True)

InPR
sub-graph's size:  4393
Number of nodes: 4393
sub-graph's size:  8786
Number of nodes: 8786
sub-graph's size:  13179
Number of nodes: 13179
sub-graph's size:  17572
Number of nodes: 17572
sub-graph's size:  21965
Number of nodes: 21965
sub-graph's size:  26358
Number of nodes: 26358
InMH
sub-graph's size:  4393
Number of nodes: 1
sub-graph's size:  8786
Number of nodes: 1
sub-graph's size:  13179
Number of nodes: 1
sub-graph's size:  17572
Number of nodes: 1
sub-graph's size:  21965
Number of nodes: 1
sub-graph's size:  26358
Number of nodes: 2042
InSRWFB
sub-graph's size:  4393
Number of nodes: 4393
sub-graph's size:  8786
Number of nodes: 8786
sub-graph's size:  13179
Number of nodes: 13179
sub-graph's size:  17572
Number of nodes: 17572
sub-graph's size:  21965
Number of nodes: 21965
sub-graph's size:  26358
Number of nodes: 26358
